In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast


In [2]:
def preprocess_data(df):
    # convert minutes
    df                       = df[df['Minutes']!=0]
    df['minutes_in_seconds'] = df['Minutes'].apply(lambda x: convert_minutes_to_sec(x))    
    df                       = df[df['lineup_labeled'].isnull()==False]
    df['lineup_labeled']     = df['lineup_labeled'].apply(lambda x: sorted(ast.literal_eval(x)))
    df['key_labels']         = df['lineup_labeled'].apply(lambda x: '-'.join(map(str, x)))
    return df

def convert_minutes_to_sec(time_str):
    try:
        minutes, seconds = time_str.split(":")
        total_seconds = int(minutes) * 60 + int(seconds)
        return total_seconds
    except Exception as e: 
        print(time_str)
        return 0

In [8]:
players = pd.read_csv('data/all_set_of_players.csv', index_col=[1,2])
players

,,id,Points,Points per player's possession,Field goals made,Field goals attempted,"Field goals, %",2-pt field goals made,2-pt field goals attempted,"2-pt field goals, %",3-pt field goals made,...,Right drives,"Right drives made, %",Left drives made,Left drives,"Left drives made, %",Opp Drives shots made,Opp Drives shots,"Opponent Drives shots made, %",minutes_in_seconds,Cluster_Labels
player_name,team_name,,,,,,,,,,,,,,,,,,,,,
Jordan McRae,Hapoel SP Tel Aviv,0,0.323011,0.098485,0.146392,1.526017,-0.355960,-0.301946,-0.116089,-0.370666,1.878927,...,0.404777,-0.932415,1.206362,1.586438,-0.183693,-0.476008,-0.239608,-0.622679,1.158308,15
Xavier Munford,Hapoel SP Tel Aviv,1,-0.119143,0.114373,-0.411176,1.060169,-0.384085,-0.998388,-0.875851,-0.277780,1.960904,...,1.030772,-0.482878,0.727194,1.327334,-0.142203,1.092025,1.051366,0.403170,1.049630,15
J'Covan Brown,Hapoel SP Tel Aviv,2,-0.445170,-0.302963,-0.743531,0.766967,-0.804552,-1.465262,-1.141860,-0.684001,2.119392,...,0.044589,-1.604921,1.030667,1.301423,0.362364,-1.006727,-1.223448,-1.174764,0.970262,15
Tomer Ginat,Hapoel SP Tel Aviv,3,-0.423616,0.317041,-0.168574,0.988058,0.043033,-0.112789,-0.083979,-0.046303,0.690916,...,0.319018,-1.445177,-0.383261,0.048262,-1.680184,0.218981,-0.562251,0.506441,0.800923,15
Idan Zalmanson,Hapoel SP Tel Aviv,4,-1.245993,-0.144148,-1.104310,0.117196,0.095209,-0.897791,-0.831268,-0.125447,0.015324,...,-0.209661,-2.554343,-0.550590,-0.218656,-1.985201,0.054711,-0.332423,0.487821,-0.253715,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nadav Malka,Hapoel Nofar Energy Galil E,203,-1.480304,-1.848220,-1.438368,-2.140527,-1.604724,-1.454546,-1.625528,-1.779390,-0.613250,...,-0.437215,0.079456,-0.398494,-0.159093,-0.880526,0.133041,-0.599437,-0.245668,-2.238457,14
Briante Weber,Bnei Herzliya,204,0.254400,0.908621,0.424738,0.676969,1.161659,0.110657,0.139128,0.907261,0.130625,...,1.472782,0.300963,0.750075,0.246842,0.018311,0.422777,0.775999,1.900004,-0.857466,6
Ron Zipper,Hapoel Altshuler Shaham Be',205,-1.373051,-1.948302,-1.832106,-1.957493,-1.836736,-1.985568,-2.095288,-1.881992,-0.222201,...,-1.834919,-0.838101,-2.075003,-2.004550,-2.383135,-1.558064,-1.388525,-1.600385,-0.773018,10


In [43]:
df = pd.read_csv('data/post_processed/lineup_data_with_id.csv')

row_game_lineups = pd.DataFrame()
# for game and team build the df of lineup columns
for game_id in df['game_id'].unique():
    filtered_df = df[df['game_id'] == game_id]
    for team_name in filtered_df['team_name'].unique():
        filtered_df = filtered_df[filtered_df['team_name'] == team_name]
        grouped_filter = filtered_df.groupby('key_labels')[['Possessions','Plus/Minus']].sum()
        total_plus_minus = grouped_filter['Plus/Minus'].sum()
        game_row = grouped_filter[['Possessions']].T
        game_row.index = [f'{game_id}_{team_name}']
        row_game_lineups = pd.concat([row_game_lineups,game_row])
row_game_lineups.fillna(0, inplace=True)

In [47]:
row_game_lineups.shape

(268, 114)